In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F

from models.args import *
from models.dataset import *
from models.discriminator import *
from models.generator import *
from models.inverter import *

from tqdm import tqdm
import warnings

In [2]:
def fxn():
    warnings.warn("deprecated", UserWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [3]:
def choose_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'

In [27]:
def train(args, train_inverter=False, num_layers=4, clamp_lower=-0.1, clamp_upper=0.1, lr=1e-3, betas=1e-5, lamb=0.1, loss_func='MSE', device=choose_device()):
    # save losses
    iloss_lst = []
    dloss_lst = []
    gloss_lst = []

    # get the dataset
    train, labels = get_dataset_with_label(args.graph_type) # entire dataset as train
    train_dataset = Graph_sequence_sampler_pytorch_nobfs(train, labels, args)
    train_loader = get_dataloader_labels(train_dataset, args)
    noise_dim = args.hidden_size_rnn
    print('noise dimension is: ', noise_dim)

    # initialize noise, optimizer and loss
    netI = Inverter(input_dim=128, output_dim=args.hidden_size_rnn, hidden_dim=64)
    netG = GraphRNN(args=args)
    netD = NetD(stat_input_dim=128, stat_hidden_dim=64, num_stat=2)

    # set up a register_hook to check parameter gradient
    for param in netD.parameters():
        h = param.register_hook(lambda grad: print("Parameter Update with gradient {:.4f}".format(grad)))

    # check model parameters
    # for param in netD.parameters():
    #     print(param.name, param.data, param.requires_grad)
    # for param in netG.parameters():
        # print(param.name, param.data, param.requires_grad)

    loss = nn.MSELoss()

    graph2vec = get_graph2vec(args.graph_type, dim=512) # use infer() to generate new graph embedding
    optimizerI = optim.Adam(netI.parameters(), lr=lr)
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=[betas for _ in range(2)])
    lossI = WGAN_ReconLoss(device, lamb, loss_func)
    G_optimizer_rnn, G_optimizer_output, G_scheduler_rnn, G_scheduler_output = netG.init_optimizer(lr=0.1) # initialize optimizers


    noise = torch.randn(args.batch_size, noise_dim).to(device)
    one = torch.tensor(1, dtype=torch.float)
    mone = torch.tensor(-1, dtype=torch.float)

    gen_iterations = 0
    for e in range(args.epochs):
        # for now, treat the input as adj matrices
        start_time = time.time()
        e_errI, e_errD, e_errG, count_batch = 0, 0, 0, 0
        for i, data in tqdm(enumerate(train_loader), desc=f"Training epoch#{e+1}", total=len(train_loader)):
            X = data['x']
            Y = data['y']
            adj_mat = data['adj_mat']
            label = data['label']
            Y_len = data['len']

            # zero grad
            optimizerI.zero_grad()
            optimizerD.zero_grad()
            G_optimizer_rnn.zero_grad()
            G_optimizer_output.zero_grad()

            # skip uneven batch
            if adj_mat.size(0) != args.batch_size:
                continue

            ######################
            # Discriminator Update
            ######################
            # number of iteration to train the discriminator
            # if gen_iterations < 25 or gen_iterations % 500 == 0:
            #     Diters = 20
            # else:
            #     Diters = 5
            Diters = 10
            j = 0 # counter for 1, 2, ... Diters

            # enable training
            netD.train(True)
            netG.train(False)
            b_errD = 0
            while j < Diters:
                j += 1
                # TODO: commenting this part out for testing
                # weight clipping: clamp parameters to a cube
                # for p in netD.parameters():
                #     p.data.clamp_(clamp_lower, clamp_upper)
                # netD.zero_grad()

                # train with real
                inputs = torch.torch.empty_like(adj_mat).copy_(adj_mat)
                D_pred = netD(inputs)
                errD_real = loss(D_pred, torch.ones(args.batch_size))
                # errD_real = Variable(torch.mean(D_pred), requires_grad=True)

                errD_real.requires_grad = True
                errD_real.retain_grad()
                errD_real.backward() # discriminator should assign 1's to true samples

                # train with fake
                # input = noise.normal_(0,1) # (batch_size, hidden_size)
                # # insert data processing
                # fake = netG(input, args, output_batch_size=args.batch_size)
                # fake_tensor = torch.Tensor([netD(nx.from_numpy_matrix(f)) for f in fake.numpy()])
                # errD_fake = Variable(torch.mean(fake_tensor), requires_grad=True)
                # errD_fake.backward(mone) # discriminator should assign -1's to fake samples??

                # # compute Wasserstein distance and update parameters
                # errD = Variable(errD_real - errD_fake, requires_grad=False)

                # print(f"Check if the model is training: iterative value at #{j}.")
                # for p in netD.parameters():
                #     print("Parameters gradients? :", p.requires_grad, end='')
                #     print("Parameters grad: ", p.grad)

                optimizerD.step()
                print(f"errD_real {errD_real.item()} ")
                # print(f"Iterative errD {errD.item()}, errD_real {errD_real.item()}, errD_fake {errD_fake.item()}: ")
                # b_errD += errD


In [5]:
args = Args()

In [14]:
# train, labels = get_dataset_with_label(args.graph_type) # entire dataset as train
# train_dataset = Graph_sequence_sampler_pytorch(train, labels, args)
# train_loader = get_dataloader_labels(train_dataset, args)
# e = 0
# for i, data in tqdm(enumerate(train_loader), desc=f"Training epoch#{e+1}", total=len(train_loader)):
#     X = data['x']
#     Y = data['y']

#     print(X.size(), Y.size())
#     print(X)
#     print('-'*50)
#     print(Y)
#     break

In [28]:
train(args=args)

noise dimension is:  128
======Generating Embedding======
======Embedding Created (used 0.6312258243560791 sec)======


Training epoch#1:   0%|          | 0/35 [00:00<?, ?it/s]

errD_real 1.2377050942660737 
errD_real 1.2377050942660737 
errD_real 1.2377050942660737 
errD_real 1.2377050942660737 
errD_real 1.2377050942660737 
errD_real 1.2377050942660737 


Training epoch#1:   0%|          | 0/35 [00:02<?, ?it/s]

errD_real 1.2377050942660737 


KeyboardInterrupt: 